In [33]:
import os
log_dir = '/vol/vssp/mammo2/will/logs/new'
fCount = 0
while os.path.exists(os.path.join(log_dir, 'training', str(fCount))):
    fCount+=1
    
print(os.path.exists(os.path.join(log_dir, 'training')))
print(log_dir + 'hello')

True
/vol/vssp/mammo2/will/logs/newhello


In [1]:
# fold
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# For some reason I have to tell it to use TensorFlows dimension ordering
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.callbacks import TensorBoard
from time import time
import os
import tensorflow as tf

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='/vol/vssp/mammo2/will/logs/new', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        fCount=0
        while os.path.exists(os.path.join(log_dir, 'training' + _ + str(fCount))):
            fCount+=1
        training_log_dir = os.path.join(log_dir, 'training' + _ + str(fCount))
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation' + _ + str(fCount))

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


def get_images(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x
 
    
    
def main():
    import keras
    print('keras version: ', keras.__version__)
    
    # Get images and labels
    data = {'img': 0, 'label': 0}
    dataSpecs = {'classLength': []}
    dataSpecs['classLength'] = []
    data['img'] = np.concatenate((
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/0/*', dataSpecs), # Class 0
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/1/*', dataSpecs) # Class 1
            get_images('/user/HS204/wm0015/student/allCalcs/0/*', dataSpecs), # Class 0
            get_images('/user/HS204/wm0015/student/allCalcs/1/*', dataSpecs) # Class 1
    ))      

    # Normalise
    data['img'] = data['img']/255    
    # Print image    
    img_calc = data['img']   
    plt.imshow(img_calc[0], cmap='gray')
    plt.show()
    
    # Create one hot labels
    labels_bg = get_labels_one_hot(2, 0, dataSpecs['classLength'][0])  
    labels_calc = get_labels_one_hot(2, 1, dataSpecs['classLength'][1])
    data['label'] = np.concatenate((
            get_labels_one_hot(2, 0, dataSpecs['classLength'][0]), # Class 0 
            get_labels_one_hot(2, 1, dataSpecs['classLength'][1]) # Class 1
    ))
    # Drop from 3 colour channels to 1 (greyscale)
    if 1==1:
        data['img'] = data['img'][:,:,:,0]
        data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
        print('new data shape = ', data['img'].shape)
    
    valStats = []
    folds = 50
    for crossVal in range(folds):

        # Shuffle data
        seed = 33
    #     np.random.seed(seed) # Has to be set before each use of random
        shuffleMask = np.random.permutation(data['img'].shape[0])    
        data['img'] = data['img'][shuffleMask, :, :, :]
        data['label'] = data['label'][shuffleMask, :]

        # Split traing and validation data        
        splitRatio = 0.9
        splitPoint = math.floor(data['img'].shape[0]*splitRatio)
        train_data = {'img': data['img'][0:splitPoint], 'label': data['label'][0:splitPoint]}
        validation_data = {'img': data['img'][splitPoint:], 'label': data['label'][splitPoint:]}
#         print('train_data[img].shape = ', train_data['img'].shape)
#         print('train_data[label].shape = ', train_data['label'].shape)
#         print('validation_data[img].shape = ', validation_data['img'].shape)
#         print('valdiation_data[label].shape = ', validation_data['label'].shape)
#         print('dataSpecs = ', dataSpecs['classLength'])


        # Keras!
        model = Sequential()

        # Layer 1
        model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        # Layer 2
        model.add(Conv2D(32, (3,3), activation='relu')) 
        model.add(MaxPooling2D(pool_size=(2, 2)))
        # Layer 3    
        model.add(Conv2D(32, (3,3), activation='relu'))  
        model.add(MaxPooling2D(pool_size=(2, 2)))
        # Layer 4     
        model.add(Conv2D(32, (3,3), activation='relu'))   
        model.add(MaxPooling2D(pool_size=(2, 2)))    
        # Layer 5     
        #model.add(Conv2D(32, (3,3), activation='relu'))   
        #model.add(MaxPooling2D(pool_size=(2, 2)))
        # Layer 6
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(2, activation='softmax'))

        sgd = optimizers.SGD(lr=5, decay=1e-6, momentum=0.9, nesterov=True)
        adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #0.001
        model.compile(loss='binary_crossentropy',
                optimizer=adam,
                metrics=['accuracy']) # Accuracy will now be returned when evaluate is called
        tensorboard = TensorBoard(log_dir='/vol/vssp/mammo2/will/logs/new'.format(time()), write_images=True)
        if folds == 1:
            model.summary()   

        # Train
        model.fit(train_data['img'], train_data['label'], 
                batch_size=100, epochs=100, verbose=1,
                callbacks=[TrainValTensorBoard(write_graph=False)],
                validation_data=(validation_data['img'], validation_data['label']))

        # Evaluate
        score = model.evaluate(validation_data['img'], validation_data['label'], verbose=0)
        valStats.append(score)
        print('The score is......\n', score)
    print(valStats,'\n')
    print('Average loss: ', sum(valStats[:][0])/folds)
    print('Average validation: ', sum(valStats[:][1])/folds)
    
if __name__ == "__main__":
    main()

Using TensorFlow backend.


keras version:  2.2.0


<Figure size 640x480 with 1 Axes>

new data shape =  (1620, 385, 385, 1)
Train on 1458 samples, validate on 162 samples
Epoch 1/100
1458/1458 [==============================] - 8s 5ms/step - loss: 0.6861 - acc: 0.5542 - val_loss: 0.6860 - val_acc: 0.4938
Epoch 2/100
1458/1458 [==============================] - 5s 3ms/step - loss: 0.6512 - acc: 0.5974 - val_loss: 0.6109 - val_acc: 0.6667
Epoch 3/100
1458/1458 [==============================] - 5s 3ms/step - loss: 0.5866 - acc: 0.6934 - val_loss: 0.5614 - val_acc: 0.6543
Epoch 4/100
1458/1458 [==============================] - 5s 3ms/step - loss: 0.4375 - acc: 0.8224 - val_loss: 0.3342 - val_acc: 0.8827
Epoch 5/100
1458/1458 [==============================] - 5s 3ms/step - loss: 0.4484 - acc: 0.7984 - val_loss: 0.4253 - val_acc: 0.8642
Epoch 6/100
1458/1458 [==============================] - 5s 4ms/step - loss: 0.2985 - acc: 0.8909 - val_loss: 0.2582 - val_acc: 0.8827
Epoch 7/100
1458/1458 [==============================] - 6s 4ms/step - loss: 0.1918 - acc: 0.9294 - val_l

1458/1458 [==============================] - 6s 4ms/step - loss: 3.4518e-05 - acc: 1.0000 - val_loss: 0.1987 - val_acc: 0.9691
Epoch 60/100
1458/1458 [==============================] - 6s 4ms/step - loss: 4.7257e-05 - acc: 1.0000 - val_loss: 0.1691 - val_acc: 0.9753
Epoch 61/100
1458/1458 [==============================] - 6s 4ms/step - loss: 8.8253e-05 - acc: 1.0000 - val_loss: 0.1485 - val_acc: 0.9753
Epoch 62/100
1458/1458 [==============================] - 6s 4ms/step - loss: 2.3013e-05 - acc: 1.0000 - val_loss: 0.1608 - val_acc: 0.9753
Epoch 63/100
1458/1458 [==============================] - 6s 4ms/step - loss: 2.8169e-05 - acc: 1.0000 - val_loss: 0.1797 - val_acc: 0.9753
Epoch 64/100
1458/1458 [==============================] - 6s 4ms/step - loss: 2.6487e-05 - acc: 1.0000 - val_loss: 0.1835 - val_acc: 0.9753
Epoch 65/100
1458/1458 [==============================] - 6s 4ms/step - loss: 2.0829e-05 - acc: 1.0000 - val_loss: 0.1919 - val_acc: 0.9691
Epoch 66/100
1458/1458 [=========

...

Epoch 72/100
1458/1458 [==============================] - 5s 3ms/step - loss: 1.5616e-05 - acc: 1.0000 - val_loss: 0.0893 - val_acc: 0.9815
Epoch 73/100
1458/1458 [==============================] - 5s 4ms/step - loss: 4.3442e-05 - acc: 1.0000 - val_loss: 0.0686 - val_acc: 0.9815
Epoch 74/100
1458/1458 [==============================] - 5s 4ms/step - loss: 3.5267e-05 - acc: 1.0000 - val_loss: 0.0566 - val_acc: 0.9815
Epoch 75/100
1458/1458 [==============================] - 5s 4ms/step - loss: 1.8710e-05 - acc: 1.0000 - val_loss: 0.0724 - val_acc: 0.9815
Epoch 76/100
1458/1458 [==============================] - 5s 4ms/step - loss: 1.6162e-05 - acc: 1.0000 - val_loss: 0.0806 - val_acc: 0.9815
Epoch 77/100
1458/1458 [==============================] - 5s 4ms/step - loss: 1.5096e-05 - acc: 1.0000 - val_loss: 0.0830 - val_acc: 0.9815
Epoch 78/100
1458/1458 [==============================] - 5s 4ms/step - loss: 1.7787e-05 - acc: 1.0000 - val_loss: 0.0725 - val_acc: 0.9815
Epoch 79/100
1458/14

1458/1458 [==============================] - 5s 3ms/step - loss: 3.3057e-04 - acc: 1.0000 - val_loss: 0.1997 - val_acc: 0.9630
Epoch 31/100
1458/1458 [==============================] - 5s 4ms/step - loss: 3.7556e-04 - acc: 1.0000 - val_loss: 0.1241 - val_acc: 0.9753
Epoch 32/100
1458/1458 [==============================] - 5s 4ms/step - loss: 4.6901e-04 - acc: 1.0000 - val_loss: 0.0800 - val_acc: 0.9753
Epoch 33/100
1458/1458 [==============================] - 5s 4ms/step - loss: 6.5326e-04 - acc: 1.0000 - val_loss: 0.2171 - val_acc: 0.9568
Epoch 34/100
1458/1458 [==============================] - 5s 4ms/step - loss: 2.6959e-04 - acc: 1.0000 - val_loss: 0.0893 - val_acc: 0.9753
Epoch 35/100
1458/1458 [==============================] - 5s 4ms/step - loss: 1.2184e-04 - acc: 1.0000 - val_loss: 0.0975 - val_acc: 0.9753
Epoch 36/100
1458/1458 [==============================] - 5s 4ms/step - loss: 1.5816e-04 - acc: 1.0000 - val_loss: 0.1354 - val_acc: 0.9753
Epoch 37/100
1458/1458 [=========

1458/1458 [==============================] - 5s 4ms/step - loss: 2.0623e-05 - acc: 1.0000 - val_loss: 0.1822 - val_acc: 0.9691
Epoch 89/100
1458/1458 [==============================] - 5s 4ms/step - loss: 1.1480e-05 - acc: 1.0000 - val_loss: 0.1449 - val_acc: 0.9753
Epoch 90/100
1458/1458 [==============================] - 5s 4ms/step - loss: 2.0682e-05 - acc: 1.0000 - val_loss: 0.1434 - val_acc: 0.9753
Epoch 91/100
1458/1458 [==============================] - 5s 4ms/step - loss: 1.6850e-05 - acc: 1.0000 - val_loss: 0.1424 - val_acc: 0.9753
Epoch 92/100
1458/1458 [==============================] - 5s 4ms/step - loss: 7.6281e-05 - acc: 1.0000 - val_loss: 0.2123 - val_acc: 0.9630
Epoch 93/100
1458/1458 [==============================] - 5s 4ms/step - loss: 4.3548e-05 - acc: 1.0000 - val_loss: 0.1606 - val_acc: 0.9753
Epoch 94/100
1458/1458 [==============================] - 5s 3ms/step - loss: 9.6298e-06 - acc: 1.0000 - val_loss: 0.1253 - val_acc: 0.9753
Epoch 95/100
1458/1458 [=========

1458/1458 [==============================] - 6s 4ms/step - loss: 8.8236e-05 - acc: 1.0000 - val_loss: 0.0265 - val_acc: 0.9938
Epoch 47/100
1458/1458 [==============================] - 5s 3ms/step - loss: 7.5880e-05 - acc: 1.0000 - val_loss: 0.0262 - val_acc: 0.9938
Epoch 48/100
1458/1458 [==============================] - 6s 4ms/step - loss: 6.2928e-05 - acc: 1.0000 - val_loss: 0.0190 - val_acc: 0.9938
Epoch 49/100
1458/1458 [==============================] - 5s 4ms/step - loss: 4.9486e-05 - acc: 1.0000 - val_loss: 0.0235 - val_acc: 0.9938
Epoch 50/100
1458/1458 [==============================] - 5s 4ms/step - loss: 5.7864e-05 - acc: 1.0000 - val_loss: 0.0349 - val_acc: 0.9815
Epoch 51/100
1458/1458 [==============================] - 6s 4ms/step - loss: 3.9333e-05 - acc: 1.0000 - val_loss: 0.0326 - val_acc: 0.9815
Epoch 52/100
1458/1458 [==============================] - 5s 4ms/step - loss: 3.3286e-05 - acc: 1.0000 - val_loss: 0.0304 - val_acc: 0.9877
Epoch 53/100
1458/1458 [=========

ResourceExhaustedError: OOM when allocating tensor with shape[100,32,383,383] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_21/Adam/gradients/max_pooling2d_85/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@max_pooling2d_85/MaxPool"], data_format="NCHW", ksize=[1, 1, 2, 2], padding="VALID", strides=[1, 1, 2, 2], _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_85/Relu, max_pooling2d_85/MaxPool, training_21/Adam/gradients/conv2d_86/convolution_grad/Conv2DBackpropInput)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_21/Adam/gradients/max_pooling2d_85/MaxPool_grad/MaxPoolGrad', defined at:
  File "/user/HS204/wm0015/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/user/HS204/wm0015/anaconda3/envs/tens

In [2]:
print('hello world')

hello world


In [12]:
x = [[0.6940992025681484, 0.3950617283950617], [0.6884314866713536, 0.5185185185185185], [0.6855149121932042, 0.5679012345679012]]
print(x[0])
print(sum(x[:][1])/2)

[0.6940992025681484, 0.3950617283950617]
0.603475002594936
